# Map of Renewable Assets in the UK

The map idea is based on: http://ukdataexplorer.com/renewables/

There is also a plan to include production data using ROC info, ELEXON (BMRS) data, or data from ENTSOE. For example as done here: http://www.renewables-map.co.uk/

Map data is from: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
ROC data would be from: https://www.renewablesandchp.ofgem.gov.uk/Public/ReportManager.aspx?ReportVisibility=1&ReportCategory=0 

In [1]:
# import libraries

import pandas as pd # for dataframes
import numpy as np  # for arrays

from pyproj import Transformer # for coordinate transformation

from bokeh.plotting import figure, output_notebook, show, output_file # for plotting map
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.palettes import Category20, inferno, plasma, viridis
from bokeh.models import WMTSTileSource
from bokeh.models import CategoricalColorMapper, Legend

output_notebook() # to plot to notebook

Loading BokehJS ...

In [2]:
# import and clean data

# data from: https://www.eia.gov/electricity/data.php#gencapacity

file_name = r'1___Utility_Y2022.xlsx'
utility = pd.read_excel(file_name,skiprows=1)

file_name = r'2___Plant_Y2022.xlsx'
plant = pd.read_excel(file_name,skiprows=1)

file_name = r'3_2_Wind_Y2022.xlsx'
wind = pd.read_excel(file_name,skiprows=1)

file_name = r'3_3_Solar_Y2022.xlsx'
solar = pd.read_excel(file_name,skiprows=1)

file_name = r'3_4_Energy_Storage_Y2022.xlsx'
storage = pd.read_excel(file_name,skiprows=1)



In [3]:
utility = utility[["Utility ID","Utility Name"]]
utility.head(5)

,Utility ID,Utility Name
0,7,"Primary Products Ingredients Americas, LLC"
1,8,Tate & Lyle Ingredients Americas Inc
2,25,Greenidge Generation Holdings LLC
3,34,City of Abbeville - (SC)
4,35,AES WR Ltd Partnership


In [4]:
plant = plant[["Utility ID","Plant Code","Latitude","Longitude"]]
plant_utility = plant.merge(utility,on="Utility ID")
plant_utility.head(5)

,Utility ID,Plant Code,Latitude,Longitude,Utility Name
0,63560,1,55.339722,-160.497222,"TDX Sand Point Generating, LLC"
1,195,2,33.458665,-87.356823,Alabama Power Co
2,195,3,31.0069,-88.0103,Alabama Power Co
3,195,4,32.583889,-86.283056,Alabama Power Co
4,195,7,34.0128,-85.9708,Alabama Power Co


In [5]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
TRANSFORM_3857_TO_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326")

In [6]:
plant_utility["Latitude"] = pd.to_numeric(plant_utility["Latitude"],errors="coerce")
plant_utility["Longitude"] = pd.to_numeric(plant_utility["Longitude"],errors="coerce")

In [7]:
plant_utility = plant_utility.dropna(subset=["Latitude","Longitude"])

In [8]:
plant_utility["x"],plant_utility["y"]=TRANSFORM_4326_TO_3857.transform(plant_utility["Latitude"], plant_utility["Longitude"])
plant_utility["coordinates"] = tuple(zip(plant_utility["Latitude"], plant_utility["Longitude"]))

In [9]:
wind = wind[["Plant Code","Plant Name","Technology","Operating Year","Nameplate Capacity (MW)","Number of Turbines","Predominant Turbine Manufacturer", "Predominant Turbine Model Number","Turbine Hub Height (Feet)"]]
wind.head(5)

,Plant Code,Plant Name,Technology,Operating Year,Nameplate Capacity (MW),Number of Turbines,Predominant Turbine Manufacturer,Predominant Turbine Model Number,Turbine Hub Height (Feet)
0,1,Sand Point,Onshore Wind Turbine,2011,0.5,1,Vestas,V39-500,155.0
1,1,Sand Point,Onshore Wind Turbine,2011,0.5,1,Vestas,V39-500,155.0
2,90,Snake River,Onshore Wind Turbine,2013,0.9,1,EWT,DW54-900,164.0
3,90,Snake River,Onshore Wind Turbine,2013,0.9,1,EWT,DW54-900,164.0
4,508,Lamar Plant,Onshore Wind Turbine,2004,4.5,3,GE,1.5 XLE,262.4


In [10]:
solar = solar[["Plant Code","Plant Name","Technology","Operating Year","Nameplate Capacity (MW)","DC Net Capacity (MW)", "Fixed Tilt?", "Azimuth Angle", "Tilt Angle","Crystalline Silicon?"]]
solar.head(5)

,Plant Code,Plant Name,Technology,Operating Year,Nameplate Capacity (MW),DC Net Capacity (MW),Fixed Tilt?,Azimuth Angle,Tilt Angle,Crystalline Silicon?
0,141,Agua Fria,Solar Photovoltaic,2001,0.2,0.1,NaN,180,33.4,Y
1,645,Big Bend,Solar Photovoltaic,2017,19.8,23,NaN,180,,NaN
2,944,Geneseo,Solar Photovoltaic,2015,1.2,1.2,Y,180,30,Y
3,960,North Ninth Street,Solar Photovoltaic,2014,0.3,0.3,Y,180,25,Y
4,1016,Butler-Warner Generation Plant,Solar Photovoltaic,2019,1.0,1,Y,180,25,Y


In [11]:
storage = storage[["Plant Code","Plant Name","Technology","Operating Year","Nameplate Capacity (MW)","Nameplate Energy Capacity (MWh)","Maximum Charge Rate (MW)", "Maximum Discharge Rate (MW)"]]
storage.head(5)

,Plant Code,Plant Name,Technology,Operating Year,Nameplate Capacity (MW),Nameplate Energy Capacity (MWh),Maximum Charge Rate (MW),Maximum Discharge Rate (MW)
0,141,Agua Fria,Batteries,2021,25.0,100,25,25
1,260,Dynegy Moss Landing Power Plant Hybrid,Batteries,2021,300.0,1200,300,300
2,260,Dynegy Moss Landing Power Plant Hybrid,Batteries,2021,100.0,400,100,100
3,389,El Centro Hybrid,Batteries,2016,30.0,20,30,30
4,645,Big Bend,Batteries,2017,12.6,26.1,12.6,12.6


In [12]:
wind_plant = wind.merge(plant_utility,on="Plant Code")
solar_plant = solar.merge(plant_utility,on="Plant Code")
storage_plant = storage.merge(plant_utility,on="Plant Code")

In [13]:
wind_plant["radius"]=2*wind_plant["Nameplate Capacity (MW)"].values**0.5
solar_plant["radius"]=2*solar_plant["Nameplate Capacity (MW)"].values**0.5
storage_plant["radius"]=2*storage_plant["Nameplate Capacity (MW)"].values**0.5

In [14]:
import re

wind_plant = wind_plant.rename(columns=lambda x: re.sub(' ','_',x))
wind_plant = wind_plant.rename(columns=lambda x: re.sub('\(','',x))
wind_plant = wind_plant.rename(columns=lambda x: re.sub('\)','',x))

solar_plant = solar_plant.rename(columns=lambda x: re.sub(' ','_',x))
solar_plant = solar_plant.rename(columns=lambda x: re.sub('\(','',x))
solar_plant = solar_plant.rename(columns=lambda x: re.sub('\)','',x))
solar_plant = solar_plant.rename(columns=lambda x: re.sub('\?','',x))

storage_plant = storage_plant.rename(columns=lambda x: re.sub(' ','_',x))
storage_plant = storage_plant.rename(columns=lambda x: re.sub('\(','',x))
storage_plant = storage_plant.rename(columns=lambda x: re.sub('\)','',x))

In [15]:
# create map plot

# See https://wiki.openstreetmap.org/wiki/Tile_servers for various tile services
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

tile_map = 'ESRI'

tools='pan,zoom_in,zoom_out,wheel_zoom,reset,save'

p = figure(tools=tools, 
        width=800, 
        height=800,
        #sizing_mode="scale_both",
        x_axis_label="longitude",
        y_axis_label="latitude",
        match_aspect=True,
        x_axis_type="mercator",
        y_axis_type="mercator")

p.add_tile(MAP_TILES[tile_map])

p.add_layout(Legend(), 'right') # neat trick to make the legend appear outside the plot (https://discourse.bokeh.org/t/adding-a-legend-outside-the-plot-area-is-possible-even-with-auto-grouped-indirectly-created-legends/5595)
    
source_wind = ColumnDataSource(wind_plant)
    
render_wind = p.scatter('x', 'y',
        size='radius', 
        alpha=0.9, 
        fill_color="cyan", 
        line_color="black",
        fill_alpha=0.3, 
        line_width=1, 
        legend_label='wind',
        source=source_wind)

# add custom hover tool
hover_wind = HoverTool(renderers=[render_wind])
hover_wind.tooltips = [("Plant Code","@Plant_Code"),
                       ("Plant Name","@Plant_Name"),
                       ("Technology","@Technology"),
                       ("Operating Year","@Operating_Year"),
                       ("Nameplate Capacity (MW)","@Nameplate_Capacity_MW"),
                       ("Number of Turbines","@Number_of_Turbines"),
                       ("Predominant Turbine Manufacturer","@Predominant_Turbine_Manufacturer"),
                       ("Predominant Turbine Model Number","@Predominant_Turbine_Model_Number"),
                       ("Turbine Hub Height (Feet)","@Turbine_Hub_Height_Feet"),]
p.add_tools(hover_wind)


source_solar = ColumnDataSource(solar_plant)
    
render_solar = p.scatter('x', 'y',
        size='radius', 
        alpha=0.9, 
        fill_color="yellow", 
        line_color="black",
        fill_alpha=0.3, 
        line_width=1, 
        legend_label='solar',
        source=source_solar)

# add custom hover tool
hover_solar = HoverTool(renderers=[render_solar])
hover_solar.tooltips = [("Plant Code","@Plant_Code"),
                       ("Plant Name","@Plant_Name"),
                       ("Technology","@Technology"),
                       ("Operating Year","@Operating_Year"),
                       ("Nameplate Capacity (MW)","@Nameplate_Capacity_MW"),
                       ("DC Net Capacity (MW)","@DC_Net_Capacity_MW"),
                       ("Fixed Tilt?","@Fixed_Tilt"),
                       ("Azimuth Angle","@Azimuth_Angle"),
                       ("Tilt Angle","@Tilt_Angle"),
                       ("Crystalline Silicon?","@Crystalline_Silicon"),]
p.add_tools(hover_solar)



source_storage = ColumnDataSource(storage_plant)
    
render_storage = p.scatter('x', 'y',
        size='radius', 
        alpha=0.9, 
        fill_color="pink", 
        line_color="black",
        fill_alpha=0.3, 
        line_width=1, 
        legend_label='storage',
        source=source_storage)

# add custom hover tool
hover_storage = HoverTool(renderers=[render_storage])
hover_storage.tooltips = [("Plant Code","@Plant_Code"),
                       ("Plant Name","@Plant_Name"),
                       ("Technology","@Technology"),
                       ("Operating Year","@Operating_Year"),
                       ("Nameplate Capacity (MW)","@Nameplate_Capacity_MW"),
                       ("Nameplate Energy Capacity (MWh)","@Nameplate_Energy_Capacity_MWh"),
                       ("Maximum Charge Rate (MW)","@Maximum_Charge_Rate_MW"),
                       ("Maximum Discharge Rate (MW)","@Maximum_Discharge_Rate_MW"),
                       ]
p.add_tools(hover_storage)

p.legend.click_policy="hide"

In [16]:
from bokeh.models import Range1d

x,y = TRANSFORM_4326_TO_3857.transform([20,50],[-130,-60])

# set a range using a Range1d
p.x_range = Range1d(x[0], x[1])
p.y_range = Range1d(y[0], y[1])

p.title.text = "USA wind, solar and storage using EIA data"

In [23]:
# show the plot
output_file(filename="Map10-NorthAmerica-USA-Renewables.html",title="USA-Renewables")

In [22]:
show(p)